In [1]:

import tensorflow as tf
import os
import chess
from environment import Board
from agent import Agent
from learning_algs import Q_learn


2023-11-29 12:11:49.305997: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# train network

environment = Board()
agent = Agent()

learn = Q_learn(agent, environment)
learn.learn(iterations=2, updateThreshold=1)

try:
    agent.saveNN()
except Exception as e:
    print(e)

In [ ]:
# play against network

board_env = Board()

while board_env.board.result() == '*':
    prediction_vector = agent.find_move(Agent.one_hot_encode(board_env.board, color=chess.WHITE))
    move = Agent.one_hot_decode(prediction_vector, board_env.board)
    board_env.play(move)
    if not(board_env.board.result() == '*'):
        break
    while True:
        move_input = input('enter move (ex: a2a3): ')
        move = chess.Move.from_uci(move_input)
        if move in board_env.board.legal_moves:
            board_env.play(move)
            break
        else:
            print('** invalid move **')

print('GAME OVER')
print('winner: ', 'white' if board_env.board.result() == '1-0' else ('black' if board_env.board.result() == '0-1' else 'draw'))
